In [ ]:
from nornir import InitNornir
from nornir_netmiko.tasks import netmiko_send_command, netmiko_send_config
from nornir_jinja2.plugins.tasks import template_file
from nornir_utils.plugins.functions import print_result
from nornir.core.task import Task, Result

In [ ]:
# Nornir loading profile
nr = InitNornir(config_file='./config.yml')
nr = InitNornir(
    runner={
        "plugin": "threaded",
        "options": {
            "num_workers": 10,
        },
    },
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "./inventory/hosts.yml",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml"
        },
    },
)
# Template catalog
template_path = './templates/'

In [ ]:
# By filtering 
sw = nr.filter(type='l2',)

In [ ]:
# Username, password
for host in sw.inventory.hosts.keys():
    nr.inventory.hosts[host].username = 'sa'
    nr.inventory.hosts[host].password = 'Qwerty11'
    nr.inventory.hosts[host].connection_options['netmiko'].extras['secret'] = 'Qwerty11'

In [ ]:
def config_sw(task:Task) -> Result:
    ios_interface_template = task.run(
        name='Get interface template',
        task=template_file,
        template='cisco_ios_interface.template',
        path=template_path
    )
    task.run(
        netmiko_send_config,
        name='Push interface config',
        config_commands=ios_interface_template.result.split('\n'),
        cmd_verify=True
    )

    return Result(
        host=task.host,
        result=f"{task.host} is running"
    )

In [ ]:
run_result = sw.run(task=config_sw,name='Configuration Switches')
print_result(run_result)

In [ ]:
dir(netmiko_send_command)